In [1]:
import pandas as pd

In [4]:
# Load data
df = pd.read_parquet(
    "../../data/experiments/label_studio_review/3_channels_predictions_09_2023_09_2024.parquet"
)
df

,start,text,channel_name,channel_is_radio,channel_program_type,channel_program,themes,keywords,num_keywords,num_tokens,claims
id,,,,,,,,,,,
1dcd4b454f8bac42440259ce26a1a2192051186bb5728be489dc654a9a967d1d,2023-09-01 06:26:00,<unk> <unk> <unk> <unk> aerosmith en janvier m...,europe1,True,Information - Magazine,Bonjour,"[""biodiversite_concepts_generaux"", ""adaptation...","[{""keyword"": ""eaux"", ""timestamp"": 169354246806...",1,383,[{'analysis': 'L'allégation semble promouvoir ...
0eb5805fa23e0819f817ea10fe1fccd19e61e40a1239cc93f701fd56bd8ea66f,2023-09-01 06:50:00,la très grande majorité d'entre eux ne connais...,europe1,True,Information - Magazine,Bonjour,"[""biodiversite_concepts_generaux"", ""changement...","[{""keyword"": ""eaux"", ""timestamp"": 169354380309...",3,593,[{'analysis': 'Cette affirmation est généralem...
b6d54aefb250671e7754a688411ce9e68badcad88a665be3de78996d13b74fd2,2023-09-01 07:38:00,mais titeuf ne vieillit pas le monde change ti...,europe1,True,Information - Magazine,Europe 1 Matin,"[""changement_climatique_constat""]","[{""keyword"": ""bilan carbone"", ""timestamp"": 169...",1,606,[{'analysis': 'L'allégation sur le fait que Ti...
23c2d3b292d9ab0fb0d0b2b8c34f3b88708c79ffab2d6659accf565ba61f48ae,2023-09-01 08:44:00,dû travailler très vite le journal arrive à no...,europe1,True,Information - Magazine,Europe 1 Matin,"[""biodiversite_concepts_generaux"", ""ressources...","[{""keyword"": ""eaux"", ""timestamp"": 169355074908...",3,515,[{'analysis': 'L'affirmation que le nucléaire ...
71df0ce2b34afa23391d8e31d35ccd213ae2a881b7ce412813a06a60a2e47d3c,2023-09-01 08:46:00,pas avoir lieu ni même européens existait ni l...,europe1,True,Information - Magazine,Europe 1 Matin,"[""biodiversite_concepts_generaux"", ""changement...","[{""keyword"": ""\u00e9cologiste"", ""timestamp"": 1...",2,536,[{'analysis': 'Cette allégation souligne un év...
...,...,...,...,...,...,...,...,...,...,...,...
121c2c2780f373e0d03ad266a5b4c11e87873e353954051693a978db123e1a04,2024-09-12 07:06:00,est la période la plus à risque pour les inond...,sud-radio,True,Information - Magazine,Le Grand Matin,"[""adaptation_climatique_solutions"", ""changemen...","[{""keyword"": ""inondation"", ""timestamp"": 172611...",2,575,[{'analysis': 'Cette déclaration peut être con...
10e4d16cad946340c67738bdaa487603d4635c6d2db978b63aff13916a49cfab,2024-09-12 08:00:00,solaire point fr votre expert en panneaux sola...,sud-radio,True,Information - Magazine,Le Grand Matin,"[""ressources_solutions"", ""changement_climatiqu...","[{""keyword"": ""panneaux solaires"", ""timestamp"":...",1,526,[{'analysis': 'Cette allégation est factuelle ...
74f1b824b772877e6e05908a308a8ae31146b652ff50793bbd7fc9d4deb5e29c,2024-09-12 09:00:00,foyer trois mille neuf cent quatre-vingt-dix e...,sud-radio,True,Information - Magazine,Le Grand Matin,"[""ressources_solutions"", ""changement_climatiqu...","[{""keyword"": ""panneaux solaires"", ""timestamp"":...",1,520,[{'analysis': 'Cette allégation est généraleme...


In [21]:
df["has_high_level_claim"] = df["claims"].apply(
    lambda claims: any(claim["disinformation_score"] == "high" for claim in claims)
)

In [25]:
df[df["has_high_level_claim"]].groupby(["channel_name"]).size()

channel_name
europe1      240
itele        898
sud-radio    422
dtype: int64

In [42]:
claims_by_quarter = pd.concat(
    [
        df[df["has_high_level_claim"]]
        .groupby(["channel_name", df["start"].dt.strftime("%Y-%m-01")])
        .size()
        .rename("has_high_level_claim"),
        df.groupby(["channel_name", df["start"].dt.strftime("%Y-%m-01")])
        .size()
        .rename("total"),
    ],
    axis=1,
)
claims_by_quarter["percentage"] = (
    (
        (claims_by_quarter["has_high_level_claim"] / claims_by_quarter["total"]) * 100
    ).round(2)
    # .astype(int)
)
claims_by_quarter

has_high_level_claim  total  percentage
channel_name start                                              
europe1      2023-09-01                  24.0    902        2.66
             2023-10-01                  18.0    751        2.40
             2023-11-01                  32.0    743        4.31
             2023-12-01                  15.0    791        1.90
             2024-01-01                  41.0    862        4.76
             2024-02-01                  44.0    768        5.73
             2024-03-01                  16.0    472        3.39
             2024-04-01                  10.0    492        2.03
             2024-05-01                  17.0    455        3.74
             2024-06-01                   8.0    418        1.91
             2024-07-01                   7.0    539        1.30
             2024-08-01                   8.0    599        1.34
itele        2023-04-01                   1.0     49        2.04
             2023-05-01                  95.0   1823        5.21
             2023-06-01                  70.0   1319        5.31
             2023-07-01                  75.0   1741        4.31
             2023-08-01                  68.0   2170        3.13
             2023-09-01                  60.0   1410        4.26
             2023-10-01                  25.0    707        3.54
             2023-11-01                  36.0   1080        3.33
             2023-12-01                  45.0    808        5.57
             2024-01-01                 148.0   1228       12.05
             2024-02-01                  87.0   1391        6.25
             2024-03-01                  33.0   1165        2.83
             2024-04-01                  40.0   1032        3.88
             2024-05-01                  31.0    755        4.11
             2024-06-01                  21.0    619        3.39
             2024-07-01                  33.0    830        3.98
             2024-08-01                  22.0    763        2.88
             2024-09-01                   8.0    217        3.69
sud-radio    2023-05-01                  40.0    678        5.90
             2023-06-01                  34.0    628        5.41
             2023-07-01                  26.0    601        4.33
             2023-08-01                  38.0    706        5.38
             2023-09-01                  28.0    422        6.64
             2023-10-01                  15.0    356        4.21
             2023-11-01                  46.0    486        9.47
             2023-12-01                  28.0    421        6.65
             2024-01-01                  26.0    511        5.09
             2024-02-01                  19.0    437        4.35
             2024-03-01                  13.0    327        3.98
             2024-04-01                  20.0    414        4.83
             2024-05-01                  30.0    409        7.33
             2024-06-01                  23.0    294        7.82
             2024-07-01                  12.0    342        3.51
             2024-08-01                  21.0    465        4.52
             2024-09-01                   3.0    117        2.56
             2023-04-01                   NaN      6         NaN

In [17]:
df["channel_name"].value_counts()

channel_name
itele        19107
europe1       7792
sud-radio     7620
Name: count, dtype: int64

In [16]:
cdf[cdf["disinformation_score"] == "high"]["id"].nunique()

59

In [7]:
df["start"] >= "2024-06-01"

id
1dcd4b454f8bac42440259ce26a1a2192051186bb5728be489dc654a9a967d1d    False
0eb5805fa23e0819f817ea10fe1fccd19e61e40a1239cc93f701fd56bd8ea66f    False
b6d54aefb250671e7754a688411ce9e68badcad88a665be3de78996d13b74fd2    False
23c2d3b292d9ab0fb0d0b2b8c34f3b88708c79ffab2d6659accf565ba61f48ae    False
71df0ce2b34afa23391d8e31d35ccd213ae2a881b7ce412813a06a60a2e47d3c    False
                                                                    ...  
121c2c2780f373e0d03ad266a5b4c11e87873e353954051693a978db123e1a04     True
10e4d16cad946340c67738bdaa487603d4635c6d2db978b63aff13916a49cfab     True
74f1b824b772877e6e05908a308a8ae31146b652ff50793bbd7fc9d4deb5e29c     True
dbb0ba82809710c830b2170abb1d30b4b0b07650b3fdadf360f92839c21a18d7     True
20c4542e60ad3387caaf54ad7cb4def5a98af162bbfb9c0f7efc5bf8c704a790     True
Name: start, Length: 34519, dtype: bool

In [ ]:
df["start"]